In [1]:
import pandas as pd
import numpy as np

# chia dữ liệu
from sklearn.model_selection import train_test_split

# tiền xử lý dữ liệu encode, scaler các thứ
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

# các thông số đánh giá mô hình
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import pickle

In [2]:
df =  pd.read_excel("..\\store\\fact.xlsx")

In [3]:
df['chenhlech_khoahoc'] = df['khoahoc'] - df['khoahoc_chuan']

In [4]:
df.columns

Index(['mssv', 'xeploai', 'namsinh', 'noisinh', 'gioitinh', 'khoa', 'hedt',
       'khoahoc', 'diem_tt', 'dtbhk_hk_1', 'dtbhk_hk_2', 'dtbhk_hk_3',
       'dtbhk_hk_4', 'dtbhk_hk_5', 'sotchk_hk_1', 'sotchk_hk_2', 'sotchk_hk_3',
       'sotchk_hk_4', 'sotchk_hk_5', 'drl_hk_1', 'drl_hk_2', 'drl_hk_3',
       'drl_hk_4', 'drl_hk_5', 'khoahoc_chuan', 'chenhlech_khoahoc'],
      dtype='object')

In [5]:
df['chenhlech_khoahoc'].value_counts()

chenhlech_khoahoc
0     1623
1      126
2       10
3        5
4        4
6        1
11       1
Name: count, dtype: int64

# Định nghĩa hàm để đánh giá

In [6]:
# Tạo results để lưu các thông số đánh giá mô hình
results_df = pd.DataFrame(columns=["Model", "Epochs", "Accuracy", "Precision", "Recall", "F1_score"])

In [7]:
def evaluate_and_save_model(name_model, epochs ,model, X_test, y_test):
    global results_df
    
    # Dự đoán từ mô hình
    y_pred = model.predict(X_test)
    if len(y_pred.shape) > 1:  # Trường hợp output là xác suất (probability)
        y_pred = y_pred.argmax(axis=1)  # Chọn lớp có xác suất cao nhất
    
    # Tính các chỉ số đánh giá
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')  # Sử dụng weighted nếu có nhiều lớp
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Lưu kết quả vào DataFrame
    new_result = pd.DataFrame({
        "Model": [name_model],
        "Epochs": [epochs],
        "Accuracy": [accuracy],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
    })
    
    results_df = pd.concat([results_df, new_result], ignore_index=True)
    return results_df

In [8]:
def evaluate_and_save_model_cnn_2_inputs(name_model, epochs, model, X_test_seq, X_test_cat, y_test):
    global results_df
    
    # Dự đoán từ mô hình (CNN với 2 đầu vào)
    y_pred = model.predict([X_test_seq, X_test_cat])
    
    if len(y_pred.shape) > 1:  # Trường hợp output là xác suất (probability)
        y_pred = y_pred.argmax(axis=1)  # Chọn lớp có xác suất cao nhất
    
    # Tính các chỉ số đánh giá
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')  # Sử dụng weighted nếu có nhiều lớp
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Lưu kết quả vào DataFrame
    new_result = pd.DataFrame({
        "Model": [name_model],
        "Epochs": [epochs],
        "Accuracy": [accuracy],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
    })
    
    # Cập nhật kết quả vào DataFrame global
    results_df = pd.concat([results_df, new_result], ignore_index=True)
    
    return results_df

In [9]:
df.drop(columns=['mssv', 'namsinh', 'khoahoc'], axis=1, inplace=True)

# MLP (Multi-Layer Perceptron)

In [10]:
# One-Hot Encoding cho dữ liệu phân loại
mlp_categorical_cols = ["noisinh", "gioitinh", "khoa", "hedt", "chenhlech_khoahoc", "diem_tt"]
mlp_encoder = OneHotEncoder()
mlp_encoded_features = mlp_encoder.fit_transform(df[mlp_categorical_cols]).toarray()

# Chuẩn hóa dữ liệu số
mlp_numerical_cols = [col for col in df.columns if "dtbhk_hk_" in col or "sotchk_hk_" in col or "drl_hk_" in col]
mlp_scaler = MinMaxScaler()
mlp_scaled_features = mlp_scaler.fit_transform(df[mlp_numerical_cols])

# Kết hợp các đặc trưng
mlp_X = np.hstack([mlp_scaled_features, mlp_encoded_features])
# Chuyển xếp loại thành nhãn số
mlp_y = df["xeploai"].astype("category").cat.codes

# Chia dữ liệu
mlp_X_train, mlp_X_temp, mlp_y_train, mlp_y_temp = train_test_split(mlp_X, mlp_y, test_size=0.2, random_state=42)
mlp_X_val, mlp_X_test, mlp_y_val, mlp_y_test = train_test_split(mlp_X_temp, mlp_y_temp, test_size=0.5, random_state=42)

# LSTM (Long Short-Term Memory)

In [11]:
# Dữ liệu chuỗi (chỉ chọn các cột tuần tự)
lstm_seq_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + [f"sotchk_hk_{i}" for i in range(1, 6)] + [f"drl_hk_{i}" for i in range(1, 6)]
# Đổi sang numpy cho phù hợp với LSTM model
lstm_df_numpy = df[lstm_seq_features].values
# Reshape: (samples, timesteps, features)
lstm_sequence_data = lstm_df_numpy.reshape(lstm_df_numpy.shape[0], 5, -1)

# Chuẩn hóa dữ liệu
lstm_scaler = MinMaxScaler()
lstm_sequence_data = lstm_sequence_data.reshape(-1, lstm_sequence_data.shape[-1])
lstm_sequence_data = lstm_scaler.fit_transform(lstm_sequence_data)
lstm_sequence_data = lstm_sequence_data.reshape(-1, 5, len(lstm_seq_features) // 5)

# Dữ liệu label
lstm_y = df["xeploai"].astype("category").cat.codes

# Chia dữ liệu
lstm_X_train, lstm_X_temp, lstm_y_train, lstm_y_temp = train_test_split(lstm_sequence_data, lstm_y, test_size=0.2, random_state=42)
lstm_X_val, lstm_X_test, lstm_y_val, lstm_y_test = train_test_split(lstm_X_temp, lstm_y_temp, test_size=0.5, random_state=42)

# CNN (Convolutional Neural Network)

In [12]:
# Dữ liệu chuỗi (chỉ chọn các cột tuần tự)
cnn_seq_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + [f"sotchk_hk_{i}" for i in range(1, 6)] + [f"drl_hk_{i}" for i in range(1, 6)]
cnn_sequence_data = df[cnn_seq_features].values
cnn_sequence_data = cnn_sequence_data.reshape(cnn_sequence_data.shape[0], 5, -1)

# Chuẩn hóa
cnn_scaler = MinMaxScaler()
cnn_sequence_data = cnn_sequence_data.reshape(-1, cnn_sequence_data.shape[-1])
cnn_sequence_data = cnn_scaler.fit_transform(cnn_sequence_data)
cnn_sequence_data = cnn_sequence_data.reshape(-1, 5, len(cnn_seq_features) // 5)

# Dữ liệu label
cnn_y = df["xeploai"].astype("category").cat.codes

# Chia dữ liệu
cnn_X_train, cnn_X_temp, cnn_y_train, cnn_y_temp = train_test_split(cnn_sequence_data, cnn_y, test_size=0.2, random_state=42)
cnn_X_val, cnn_X_test, cnn_y_val, cnn_y_test = train_test_split(cnn_X_temp, cnn_y_temp, test_size=0.5, random_state=42)

# LSTM 2 inputs

In [13]:
# Trích xuất các đặc trưng
lstm_2_inputs_categorical_features = ["noisinh", "gioitinh", "khoa", "hedt", "chenhlech_khoahoc", "diem_tt"]
lstm_2_inputs_sequence_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + \
                                   [f"sotchk_hk_{i}" for i in range(1, 6)] + \
                                   [f"drl_hk_{i}" for i in range(1, 6)]

# Xử lý các đặc trưng categorical
lstm_2_inputs_cat_data = df[lstm_2_inputs_categorical_features].copy()
for col in lstm_2_inputs_categorical_features:
    le = LabelEncoder()
    lstm_2_inputs_cat_data[col] = le.fit_transform(lstm_2_inputs_cat_data[col])

# Chuẩn hóa dữ liệu categorical
lstm_2_inputs_cat_data = MinMaxScaler().fit_transform(lstm_2_inputs_cat_data)

# Xử lý dữ liệu sequence
lstm_2_inputs_sequence_data = df[lstm_2_inputs_sequence_features].values
lstm_2_inputs_sequence_data = lstm_2_inputs_sequence_data.reshape(lstm_2_inputs_sequence_data.shape[0], 5, -1)
lstm_2_inputs_scaler = MinMaxScaler()
lstm_2_inputs_sequence_data = lstm_2_inputs_sequence_data.reshape(-1, lstm_2_inputs_sequence_data.shape[-1])
lstm_2_inputs_sequence_data = lstm_2_inputs_scaler.fit_transform(lstm_2_inputs_sequence_data)
lstm_2_inputs_sequence_data = lstm_2_inputs_sequence_data.reshape(-1, 5, len(lstm_2_inputs_sequence_features) // 5)

# Chia dữ liệu
lstm_2_inputs_y = df["xeploai"].astype("category").cat.codes
lstm_2_inputs_X_train_seq, lstm_2_inputs_X_temp_seq, lstm_2_inputs_y_train, lstm_2_inputs_y_temp = train_test_split(
    lstm_2_inputs_sequence_data, lstm_2_inputs_y, test_size=0.2, random_state=42)
lstm_2_inputs_X_train_cat, lstm_2_inputs_X_temp_cat = train_test_split(
    lstm_2_inputs_cat_data, test_size=0.2, random_state=42)
lstm_2_inputs_X_val_seq, lstm_2_inputs_X_test_seq, lstm_2_inputs_y_val, lstm_2_inputs_y_test = train_test_split(
    lstm_2_inputs_X_temp_seq, lstm_2_inputs_y_temp, test_size=0.5, random_state=42)
lstm_2_inputs_X_val_cat, lstm_2_inputs_X_test_cat = train_test_split(
    lstm_2_inputs_X_temp_cat, test_size=0.5, random_state=42)

# CNN 2 inputs

In [14]:
# Trích xuất các đặc trưng
cnn_2_inputs_categorical_features = ["noisinh", "gioitinh", "khoa", "hedt", "chenhlech_khoahoc", "diem_tt"]
cnn_2_inputs_sequence_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + \
                                  [f"sotchk_hk_{i}" for i in range(1, 6)] + \
                                  [f"drl_hk_{i}" for i in range(1, 6)]

# Xử lý các đặc trưng categorical
cnn_2_inputs_cat_data = df[cnn_2_inputs_categorical_features].copy()
cnn_2_inputs_label_encoders = {}
for col in cnn_2_inputs_categorical_features:
    le = LabelEncoder()
    cnn_2_inputs_cat_data[col] = le.fit_transform(cnn_2_inputs_cat_data[col])
    cnn_2_inputs_label_encoders[col] = le

# Chuẩn hóa categorical data
cnn_2_inputs_cat_data = MinMaxScaler().fit_transform(cnn_2_inputs_cat_data)

# Huấn luyện scaler_cat
cnn_2_inputs_scaler_cat = MinMaxScaler()
cnn_2_inputs_scaler_cat.fit(cnn_2_inputs_cat_data)

cnn_2_inputs_sequence_data = df[cnn_2_inputs_sequence_features].values
cnn_2_inputs_sequence_data = cnn_2_inputs_sequence_data.reshape(
    cnn_2_inputs_sequence_data.shape[0], 5, -1)  # (n_samples, 5, n_features_per_step)

# Chuyển sequence_data thành 2D để fit MinMaxScaler
cnn_2_inputs_scaler_seq = MinMaxScaler()
cnn_2_inputs_sequence_data_2d = cnn_2_inputs_sequence_data.reshape(
    -1, cnn_2_inputs_sequence_data.shape[-1])  # (n_samples * 5, n_features_per_step)

# Huấn luyện scaler
cnn_2_inputs_scaler_seq.fit(cnn_2_inputs_sequence_data_2d)

# Chuẩn hóa sequence_data và chuyển lại về 3D
cnn_2_inputs_sequence_data_2d_scaled = cnn_2_inputs_scaler_seq.transform(
    cnn_2_inputs_sequence_data_2d)  # Chuẩn hóa dữ liệu 2D
cnn_2_inputs_sequence_data_scaled = cnn_2_inputs_sequence_data_2d_scaled.reshape(
    -1, 5, cnn_2_inputs_sequence_data.shape[-1])  # Chuyển về 3D

# Chia dữ liệu
cnn_2_inputs_y = df["xeploai"].astype("category").cat.codes
cnn_2_inputs_X_train_seq, cnn_2_inputs_X_temp_seq, cnn_2_inputs_y_train, cnn_2_inputs_y_temp = train_test_split(
    cnn_2_inputs_sequence_data_scaled, cnn_2_inputs_y, test_size=0.2, random_state=42)
cnn_2_inputs_X_train_cat, cnn_2_inputs_X_temp_cat = train_test_split(
    cnn_2_inputs_cat_data, test_size=0.2, random_state=42)
cnn_2_inputs_X_val_seq, cnn_2_inputs_X_test_seq, cnn_2_inputs_y_val, cnn_2_inputs_y_test = train_test_split(
    cnn_2_inputs_X_temp_seq, cnn_2_inputs_y_temp, test_size=0.5, random_state=42)
cnn_2_inputs_X_val_cat, cnn_2_inputs_X_test_cat = train_test_split(
    cnn_2_inputs_X_temp_cat, test_size=0.5, random_state=42)

# chạy mô hình

In [15]:
for epochs in range(25, 300, 25):
    print(f"\nTraining models with epochs: {epochs}")

    # Tạo lại MLP model
    mlp_model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(mlp_X_train.shape[1],)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(len(np.unique(mlp_y)), activation='softmax')
    ])
    mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print("Training MLP...")
    mlp_history = mlp_model.fit(mlp_X_train, mlp_y_train, validation_data=(mlp_X_val, mlp_y_val), epochs=epochs, batch_size=32)
    evaluate_and_save_model("MLP - Multi-Layer Perceptron e:" + str(epochs), epochs, mlp_model, mlp_X_test, mlp_y_test)

    # Tạo lại LSTM model
    lstm_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(5, len(lstm_seq_features) // 5), return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(len(np.unique(lstm_y)), activation='softmax')
    ])
    lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print("Training LSTM...")
    lstm_history = lstm_model.fit(lstm_X_train, lstm_y_train, validation_data=(lstm_X_val, lstm_y_val), epochs=epochs, batch_size=32)
    evaluate_and_save_model("LSTM (Long Short-Term Memory) e:" + str(epochs), epochs, lstm_model, lstm_X_test, lstm_y_test)

    # Tạo lại CNN model
    cnn_model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(5, len(cnn_seq_features) // 5)),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(64, kernel_size=2, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(len(np.unique(cnn_y)), activation='softmax')
    ])
    cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print("Training CNN...")
    cnn_history = cnn_model.fit(cnn_X_train, cnn_y_train, validation_data=(cnn_X_val, cnn_y_val), epochs=epochs, batch_size=32)
    evaluate_and_save_model("CNN (Convolutional Neural Network) e:" + str(epochs), epochs, cnn_model, cnn_X_test, cnn_y_test)

    # Tạo lại LSTM với 2 đầu vào model
    lstm_2_inputs_sequence_input = tf.keras.Input(shape=(5, len(lstm_2_inputs_sequence_features) // 5))
    lstm_2_inputs_x_seq = tf.keras.layers.LSTM(64, return_sequences=True)(lstm_2_inputs_sequence_input)
    lstm_2_inputs_x_seq = tf.keras.layers.Dropout(0.3)(lstm_2_inputs_x_seq)
    lstm_2_inputs_x_seq = tf.keras.layers.LSTM(32)(lstm_2_inputs_x_seq)
    lstm_2_inputs_x_seq = tf.keras.layers.Dropout(0.3)(lstm_2_inputs_x_seq)

    # Nhánh xử lý dữ liệu categorical
    lstm_2_inputs_categorical_input = tf.keras.Input(shape=(lstm_2_inputs_cat_data.shape[1],))
    lstm_2_inputs_x_cat = tf.keras.layers.Dense(64, activation='relu')(lstm_2_inputs_categorical_input)
    lstm_2_inputs_x_cat = tf.keras.layers.Dropout(0.3)(lstm_2_inputs_x_cat)

    # Kết hợp cả hai nhánh
    lstm_2_inputs_combined = tf.keras.layers.concatenate([lstm_2_inputs_x_seq, lstm_2_inputs_x_cat])

    # Các tầng Dense cuối cùng để dự đoán
    lstm_2_inputs_x = tf.keras.layers.Dense(64, activation='relu')(lstm_2_inputs_combined)
    lstm_2_inputs_x = tf.keras.layers.Dropout(0.3)(lstm_2_inputs_x)
    lstm_2_inputs_output = tf.keras.layers.Dense(len(np.unique(lstm_2_inputs_y)), activation='softmax')(lstm_2_inputs_x)

    lstm_2_inputs_model = tf.keras.Model(
        inputs=[lstm_2_inputs_sequence_input, lstm_2_inputs_categorical_input], 
        outputs=lstm_2_inputs_output)
    lstm_2_inputs_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print("Training LSTM with 2 Inputs...")
    lstm_2_inputs_history = lstm_2_inputs_model.fit([lstm_2_inputs_X_train_seq, lstm_2_inputs_X_train_cat], lstm_2_inputs_y_train, 
                                                    validation_data=([lstm_2_inputs_X_val_seq, lstm_2_inputs_X_val_cat], lstm_2_inputs_y_val), 
                                                    epochs=epochs, batch_size=32)
    evaluate_and_save_model_cnn_2_inputs("LSTM (Long Short-Term Memory) 2 Inputs e:" + str(epochs), epochs, lstm_2_inputs_model, lstm_2_inputs_X_test_seq, lstm_2_inputs_X_test_cat, lstm_2_inputs_y_test)

    # Tạo lại CNN với 2 đầu vào model
    cnn_2_inputs_sequence_input = tf.keras.Input(shape=(5, len(cnn_2_inputs_sequence_features) // 5))
    cnn_2_inputs_x_seq = tf.keras.layers.Conv1D(32, kernel_size=2, activation='relu')(cnn_2_inputs_sequence_input)
    cnn_2_inputs_x_seq = tf.keras.layers.MaxPooling1D(pool_size=2)(cnn_2_inputs_x_seq)
    cnn_2_inputs_x_seq = tf.keras.layers.Conv1D(64, kernel_size=2, activation='relu')(cnn_2_inputs_x_seq)
    cnn_2_inputs_x_seq = tf.keras.layers.GlobalMaxPooling1D()(cnn_2_inputs_x_seq)

    # Nhánh xử lý categorical
    cnn_2_inputs_categorical_input = tf.keras.Input(shape=(cnn_2_inputs_cat_data.shape[1],))
    cnn_2_inputs_x_cat = tf.keras.layers.Dense(64, activation='relu')(cnn_2_inputs_categorical_input)

    # Kết hợp các nhánh
    cnn_2_inputs_combined = tf.keras.layers.concatenate([cnn_2_inputs_x_seq, cnn_2_inputs_x_cat])
    cnn_2_inputs_x = tf.keras.layers.Dense(64, activation='relu')(cnn_2_inputs_combined)
    cnn_2_inputs_x = tf.keras.layers.Dropout(0.3)(cnn_2_inputs_x)
    cnn_2_inputs_output = tf.keras.layers.Dense(len(np.unique(cnn_2_inputs_y)), activation='softmax')(cnn_2_inputs_x)

    cnn_2_inputs_model = tf.keras.Model(inputs=[cnn_2_inputs_sequence_input, cnn_2_inputs_categorical_input], 
                                        outputs=cnn_2_inputs_output)
    cnn_2_inputs_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print("Training CNN with 2 Inputs...")
    cnn_2_inputs_history = cnn_2_inputs_model.fit([cnn_2_inputs_X_train_seq, cnn_2_inputs_X_train_cat], cnn_2_inputs_y_train, 
                                                  validation_data=([cnn_2_inputs_X_val_seq, cnn_2_inputs_X_val_cat], cnn_2_inputs_y_val), 
                                                  epochs=epochs, batch_size=32)
    evaluate_and_save_model_cnn_2_inputs("CNN (Convolutional Neural Network) 2 Inputs e:" + str(epochs), epochs, cnn_2_inputs_model, cnn_2_inputs_X_test_seq, cnn_2_inputs_X_test_cat, cnn_2_inputs_y_test)


Training models with epochs: 25
Training MLP...


E:\Anaconda 3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4838 - loss: 1.2258 - val_accuracy: 0.6441 - val_loss: 0.8960
Epoch 2/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6211 - loss: 0.9272 - val_accuracy: 0.6667 - val_loss: 0.8342
Epoch 3/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6402 - loss: 0.8701 - val_accuracy: 0.6554 - val_loss: 0.7808
Epoch 4/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6526 - loss: 0.8174 - val_accuracy: 0.6554 - val_loss: 0.7233
Epoch 5/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6630 - loss: 0.7558 - val_accuracy: 0.6723 - val_loss: 0.6922
Epoch 6/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6600 - loss: 0.7406 - val_accuracy: 0.6780 - val_loss: 0.6615
Epoch 7/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7293 - loss: 0.6545 - val_accuracy: 0.7175 - val_loss: 0.6313
Epoch 8/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7291 - loss: 0.6268 - val_accuracy: 0.7119 - val_loss:

E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anhthy\AppData\Local\Temp\ipykernel_6584\272411793.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_result], ignore_index=True)
E:\Anaconda 3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super()._

Epoch 1/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.5453 - loss: 1.2475 - val_accuracy: 0.6441 - val_loss: 0.9202
Epoch 2/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6072 - loss: 0.9782 - val_accuracy: 0.6441 - val_loss: 0.9017
Epoch 3/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6089 - loss: 0.9666 - val_accuracy: 0.6441 - val_loss: 0.8989
Epoch 4/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6079 - loss: 0.9669 - val_accuracy: 0.6441 - val_loss: 0.9064
Epoch 5/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6082 - loss: 0.9580 - val_accuracy: 0.6441 - val_loss: 0.9018
Epoch 6/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6018 - loss: 0.9581 - val_accuracy: 0.6441 - val_loss: 0.8897
Epoch 7/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6178 - loss: 0.9299 - val_accuracy: 0.6441 - val_loss: 0.8915
Epoch 8/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6106 - loss: 0.9584 - val_accuracy: 0.6441 - val_loss

...

E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Anaconda 3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_450', 'keras_tensor_455']. Received: the structure of inputs=('*', '*')
  warnings.warn(


45/45 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.4364 - loss: 1.2788 - val_accuracy: 0.6441 - val_loss: 0.9454
Epoch 2/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5964 - loss: 1.0168 - val_accuracy: 0.6441 - val_loss: 0.9031
Epoch 3/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6116 - loss: 0.9813 - val_accuracy: 0.6441 - val_loss: 0.9040
Epoch 4/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6043 - loss: 0.9849 - val_accuracy: 0.6441 - val_loss: 0.9036
Epoch 5/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6123 - loss: 0.9628 - val_accuracy: 0.6441 - val_loss: 0.8953
Epoch 6/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6269 - loss: 0.9549 - val_accuracy: 0.6441 - val_loss: 0.8988
Epoch 7/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6012 - loss: 0.9822 - val_accuracy: 0.6441 - val_loss: 0.8967
Epoch 8/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6010 - loss: 0.9564 - val_accuracy: 0.6441 - val_loss: 0.

E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Anaconda 3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_462', 'keras_tensor_467']. Received: the structure of inputs=('*', '*')
  warnings.warn(


45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5858 - loss: 1.1233 - val_accuracy: 0.6441 - val_loss: 0.9211
Epoch 2/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6076 - loss: 0.9794 - val_accuracy: 0.6441 - val_loss: 0.9055
Epoch 3/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6035 - loss: 0.9535 - val_accuracy: 0.6441 - val_loss: 0.8872
Epoch 4/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6105 - loss: 0.9326 - val_accuracy: 0.6441 - val_loss: 0.8844
Epoch 5/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6299 - loss: 0.9326 - val_accuracy: 0.6441 - val_loss: 0.8780
Epoch 6/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6219 - loss: 0.9011 - val_accuracy: 0.6441 - val_loss: 0.8549
Epoch 7/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6084 - loss: 0.8835 - val_accuracy: 0.6441 - val_loss: 0.8347
Epoch 8/275
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6235 - loss: 0.8369 - val_accuracy: 0.6554 - val_loss: 0.7

E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# so sánh các mô hình

In [16]:
results_df

,Model,Epochs,Accuracy,Precision,Recall,F1_score
0,MLP - Multi-Layer Perceptron e:25,25,0.723164,0.736200,0.723164,0.709250
1,LSTM (Long Short-Term Memory) e:25,25,0.757062,0.742760,0.757062,0.746712
2,CNN (Convolutional Neural Network) e:25,25,0.706215,0.699159,0.706215,0.685687
3,LSTM (Long Short-Term Memory) 2 Inputs e:25,25,0.717514,0.728339,0.717514,0.712608
4,CNN (Convolutional Neural Network) 2 Inputs e:25,25,0.768362,0.772729,0.768362,0.759583
5,MLP - Multi-Layer Perceptron e:50,50,0.779661,0.781386,0.779661,0.772771
6,LSTM (Long Short-Term Memory) e:50,50,0.728814,0.726896,0.728814,0.726809
7,CNN (Convolutional Neural Network) e:50,50,0.813559,0.805298,0.813559,0.804496
8,LSTM (Long Short-Term Memory) 2 Inputs e:50,50,0.740113,0.736887,0.740113,0.737144
9,CNN (Convolutional Neural Network) 2 Inputs e:50,50,0.830508,0.819834,0.830508,0.820696


In [17]:
results_df.to_excel("..\store\\Epochs Test Deep Learning.xlsx", index=False)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\anhthy\AppData\Local\Temp\ipykernel_6584\2825833869.py:1: SyntaxWarning: invalid escape sequence '\s'
  results_df.to_excel("..\store\\Epochs Test Deep Learning.xlsx", index=False)
